# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging

from matplotlib import pyplot as plt
import pandas as pd
import os

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.automl.core.featurization import FeaturizationConfig
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
# choose a name for experiment
experiment_name = 'capstone-automl'
project_folder = './capstone-project3'

experiment=Experiment(ws, experiment_name)

Workspace name: quick-starts-ws-134658
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-134658


In [3]:
from azureml.core import Environment
# set up the  environment
env = Environment.from_conda_specification(
        name='cap-env',
        file_path='./cap-env.yml'    
        )

## Create or Attach existing AmlCompute

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "aml-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Placement Dataset"
description_text = "MBA placement dataset for Capstone Project"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/webpagearshi/capstone-project/master/starter_file/placement_data_mba.csv'
        ds = TabularDatasetFactory.from_delimited_files(path = example_data)        
        #Register Dataset in Workspace
        ds = ds.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = ds.to_pandas_dataframe()
df.describe()

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,148.000000
mean,108.000000,67.303395,66.333163,66.370186,72.100558,62.278186,288655.405405
std,62.209324,10.827205,10.897509,7.358743,13.275956,5.833385,93457.452420
min,1.000000,40.890000,37.000000,50.000000,50.000000,51.210000,200000.000000
25%,54.500000,60.600000,60.900000,61.000000,60.000000,57.945000,240000.000000
50%,108.000000,67.000000,65.000000,66.000000,71.000000,62.000000,265000.000000
75%,161.500000,75.700000,73.000000,72.000000,83.500000,66.255000,300000.000000
max,215.000000,89.400000,97.700000,91.000000,98.000000,77.890000,940000.000000


In [6]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [7]:
from sklearn.model_selection import train_test_split
#create train test split
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.2, random_state=223)
#join the train_x and train_y to create train dataset
train_df=pd.concat([train_x,train_y], axis=1)

In [8]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
# Save the train data to a csv to be uploaded to the datastore
pd.DataFrame(train_df).to_csv("data/train_data.csv", index=False)

ds = ws.get_default_datastore()
ds.upload(src_dir='./data', target_path='mba_placement', overwrite=True, show_progress=True)

 

# Upload the training data as a tabular dataset for access during training on remote compute
train_data = TabularDatasetFactory.from_delimited_files(path=ds.path('mba_placement/train_data.csv'))
label = "status"

Uploading an estimated of 1 files
Uploading ./data/train_data.csv
Uploaded ./data/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_hours" : 0.3,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=train_data,
                             label_column_name=label,   
                             path = project_folder,
                             debug_log = "automl_errors.log",
                             **automl_settings
)

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
from azureml.core.run import Run
###model_explainability_run_id = remote_run.id + "_" + "ModelExplain"
#print(model_explainability_run_id)
##model_explainability_run = Run(experiment=experiment, run_id=model_explainability_run_id)
#model_explainability_run.wait_for_completion()

# Get the best run object
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
best_run.get_tags()
best_run.properties['model_name']

Run(Experiment: capstone-automl,
Id: AutoML_89b76151-e951-4e1a-8b3e-9276d9a6e6cf_4,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MinMaxScaler', MinMaxScaler(copy...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight='balanced',
                                        criterion='gini', max_depth=None,
                                        max_features=

'AutoML89b76151e4'

In [17]:
#from azureml.automl.core.shared import constants
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'cap-env.yml')

In [13]:
#TODO: Save the best model
import joblib
os.makedirs('outputs', exist_ok = True)
joblib.dump(fitted_model, 'outputs/fitted_model.joblib')

['outputs/fitted_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
best_run, fitted_model = remote_run.get_output()
model_name = best_run.properties['model_name']

script_file_name = './score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', './score.py')


In [15]:
best_run.register_model("auto-ml" , model_path = './outputs/')

Model(workspace=Workspace.create(name='quick-starts-ws-134658', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-134658'), name=auto-ml, id=auto-ml:1, version=1, tags={}, properties={})

In [16]:
description = 'AutoML Model trained on MBA Placement data to predict if a student gets placed or not.'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id) # This will be written to the script file later in the notebook.

AutoML89b76151e4


TODO: In the cell below, send a request to the web service you deployed to test it.

In [18]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

#myenv = Environment.get(ws, name="cap-env", version=None)
inference_config = InferenceConfig(entry_script=script_file_name, environment=env)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1,
                                               enable_app_insights=True, 
                                               tags = {'area': "PlacementData", 'type': "automl_classification"}, 
                                               description = 'sample service for Automl Classification')

aci_service_name = 'automl-mba-placement-all'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

automl-mba-placement-all
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running........................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [19]:
print(aci_service.scoring_uri)

http://c3c2e2db-cbd1-4407-8dc7-7bc2df423060.southcentralus.azurecontainer.io/score


In [20]:
# Load the mba placement datasets.
import numpy as np
from numpy import array

In [21]:
import json
import requests

test_x_json = test_x.to_json(orient='records')
data = "{\"data\": " + test_x_json +"}"
headers = {'Content-Type': 'application/json'}

resp = requests.post(aci_service.scoring_uri, data, headers=headers)

y_pred = json.loads(json.loads(resp.text))['result']

In [22]:
from sklearn.metrics import confusion_matrix

conf_mx = confusion_matrix(test_y, y_pred)
print(conf_mx)
print('Overall accuracy:', np.average(y_pred == test_y))

[[ 9  6]
 [ 1 27]]
Overall accuracy: 0.8372093023255814


In [25]:
actual = array(test_y)
actual = actual[:,0]
print(len(y_pred), " ", len(actual))

IndexError: too many indices for array

TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
aci_service.get_logs()

'2021-01-14T13:38:01,242283600+00:00 - iot-server/run \n2021-01-14T13:38:01,246884500+00:00 - nginx/run \n2021-01-14T13:38:01,244959700+00:00 - rsyslog/run \n/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_9c539d20199ae6be65c41c0382029684/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-01-14T13:38:01,263287700+00:00 - gunicorn/run 

In [ ]:
aci_service.delete()